In [150]:
import os
import json

def load_json_files(folder_path):
    data = []
    for folder_name in os.listdir(folder_path):
        folder_full_path = os.path.join(folder_path, folder_name)
        if os.path.isdir(folder_full_path):
            print(folder_full_path)
            files = os.listdir(folder_full_path)

            for file_name in files:
                if file_name.endswith('.json'):
                    file_path = os.path.join(folder_full_path, file_name)
                    dataset = {
                        'model' : folder_name.replace("_image" , ""),
                        'dataset' : file_name.replace("_results.json" , ""),
                        'results' : None
                    }

                    with open(file_path, 'r') as f:
                        dataset['results'] = json.load(f)

                    data.append(dataset)
    return data

# Load all JSON files in the 'res' folder
folder_path = './res/'
data = load_json_files(folder_path)

for d in data:
    print(d['model'], d['dataset'], len(d['results']))
    print(d['results'][0])


./res/unirare_exp2_finetune_threshold_0.8_image
./res/unirare_finetune_threshold_0.8_image
./res/unirare_exp_finetune_threshold_0.7_image
./res/unirare_exp_threshold_0.7_image
./res/deeprare_0.7
./res/deeprare_noThreshold
unirare_exp2_finetune_threshold_0.8 O3_data 2001
{'filename': 'v62pUIknqAPt0.jpg', 'path': '/Users/coconut/Documents/Dataset/Saliency/O3_data/images/', 'metrics': {'saliency': {'msrt': 0.2392, 'msrb': 3.377}, 'saliency_rare': {'msrt': 1.4394, 'msrb': 1.3316}, 'saliency_fusion_add': {'msrt': 0.5333, 'msrb': 1.5}, 'saliency_fusion_rs': {'msrt': 1.1848, 'msrb': 1.1697}}}
unirare_exp2_finetune_threshold_0.8 P3_data_orientations 864
{'filename': 'image_ellipse_45_125.png', 'path': '/Users/coconut/Documents/Dataset/Saliency/P3_data/orientations/images/', 'metrics': {'saliency': {'msrt': 0.051, 'msrb': 19.6154}, 'saliency_rare': {'msrt': 4.2931, 'msrb': 1.0161}, 'saliency_fusion_add': {'msrt': 0.9724, 'msrb': 1.0283}, 'saliency_fusion_rs': {'msrt': 0.9843, 'msrb': 1.02}}}
un

In [151]:
import numpy as np

def calculate_mean(data,key):
    msrts, msrbs = [], []
    for item in data:
        msrts.append(item['metrics'][key]['msrt'])
        msrbs.append(item['metrics'][key]['msrb'])  
    msrts = np.asarray(msrts)
    msrbs = np.asarray(msrbs)
    mean_msrt = msrts.sum() / len(msrts)
    mean_msrb = msrbs.sum() / len(msrbs)
    return float(round(mean_msrt,2)), float(round(mean_msrb,2))


results = []
for d in data:
    # get list of all keys resulst
    keys = list(d['results'][0]['metrics'].keys())
    print(keys)


    for key in keys:
        msrt, msrb = calculate_mean(d['results'],key)
        filename= filename.replace('_image' , '')
        filename= filename.replace('results_' , '')

        result= {
            'model' : d['model'],
            'dataset' : d['dataset'],
            'key' : key,
            'metrics' :  {
                'msrt': msrt,
                'msrb': msrb
            }
        }   

        print(result)

        results.append(result)

    # break

['saliency', 'saliency_rare', 'saliency_fusion_add', 'saliency_fusion_rs']
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'O3_data', 'key': 'saliency', 'metrics': {'msrt': 0.81, 'msrb': 1.95}}
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'O3_data', 'key': 'saliency_rare', 'metrics': {'msrt': 1.21, 'msrb': 1.13}}
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'O3_data', 'key': 'saliency_fusion_add', 'metrics': {'msrt': 1.02, 'msrb': 0.97}}
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'O3_data', 'key': 'saliency_fusion_rs', 'metrics': {'msrt': 0.84, 'msrb': 1.26}}
['saliency', 'saliency_rare', 'saliency_fusion_add', 'saliency_fusion_rs']
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'P3_data_orientations', 'key': 'saliency', 'metrics': {'msrt': 0.14, 'msrb': 61.71}}
{'model': 'unirare_exp2_finetune_threshold_0.8', 'dataset': 'P3_data_orientations', 'key': 'saliency_rare', 'metrics': {'msrt': 2.91, 'msrb': 1.03}}
{'mode

In [152]:
from IPython.display import display, HTML
# get list of datasets from results
datasets_name = list(set([result['dataset'] for result in results]))
datasets_results = {dataset : [] for dataset in datasets_name}

print(datasets_results)

for result in results:
    datasets_results[result['dataset']].append(result)

import pandas as pd
for dataset, data in datasets_results.items():
    print(dataset)
    models = []
    types = []
    msrts= []
    msrbs= []
    for res in data:
        name = res['model'] + " " + res['key']
        msrt = res['metrics']['msrt']
        msrb = res['metrics']['msrb']

        models.append(res['model'])
        types.append(res['key'])

        msrts.append(msrt)
        msrbs.append(msrb)
    # show tables results of names msrts and msrbs
    table_data = {
        'Models': models,
        'Type': types,
        'MSRT': msrts,
        'MSRB': msrbs
    }
    df = pd.DataFrame(table_data)
    styled_df = df.style.background_gradient(subset=['MSRT', 'MSRB'], cmap='coolwarm')
    display(styled_df)




{'P3_data_colors': [], 'O3_data': [], 'P3_data_sizes': [], 'P3_data_orientations': []}
P3_data_colors


,Models,Type,MSRT,MSRB
0,unirare_exp2_finetune_threshold_0.8,saliency,0.300000,17.580000
1,unirare_exp2_finetune_threshold_0.8,saliency_rare,5.150000,0.880000
2,unirare_exp2_finetune_threshold_0.8,saliency_fusion_add,1.140000,1.020000
3,unirare_exp2_finetune_threshold_0.8,saliency_fusion_rs,0.850000,1.290000
4,unirare_finetune_threshold_0.8,saliency,0.300000,20.780000
5,unirare_finetune_threshold_0.8,saliency_rare,3.300000,0.920000
6,unirare_finetune_threshold_0.8,saliency_fusion_add,1.090000,1.060000
7,unirare_finetune_threshold_0.8,saliency_fusion_rs,0.920000,1.220000
8,unirare_exp_finetune_threshold_0.7,saliency,0.300000,17.580000
9,unirare_exp_finetune_threshold_0.7,saliency_rare,4.530000,0.900000


O3_data


,Models,Type,MSRT,MSRB
0,unirare_exp2_finetune_threshold_0.8,saliency,0.810000,1.950000
1,unirare_exp2_finetune_threshold_0.8,saliency_rare,1.210000,1.130000
2,unirare_exp2_finetune_threshold_0.8,saliency_fusion_add,1.020000,0.970000
3,unirare_exp2_finetune_threshold_0.8,saliency_fusion_rs,0.840000,1.260000
4,unirare_finetune_threshold_0.8,saliency,0.800000,1.850000
5,unirare_finetune_threshold_0.8,saliency_rare,1.100000,1.020000
6,unirare_finetune_threshold_0.8,saliency_fusion_add,0.990000,0.950000
7,unirare_finetune_threshold_0.8,saliency_fusion_rs,0.860000,1.210000
8,unirare_exp_finetune_threshold_0.7,saliency,0.810000,1.900000
9,unirare_exp_finetune_threshold_0.7,saliency_rare,1.180000,1.090000


P3_data_sizes


,Models,Type,MSRT,MSRB
0,unirare_exp2_finetune_threshold_0.8,saliency,0.130000,37.610000
1,unirare_exp2_finetune_threshold_0.8,saliency_rare,1.790000,1.150000
2,unirare_exp2_finetune_threshold_0.8,saliency_fusion_add,0.760000,1.480000
3,unirare_exp2_finetune_threshold_0.8,saliency_fusion_rs,0.920000,1.290000
4,unirare_finetune_threshold_0.8,saliency,0.130000,30.760000
5,unirare_finetune_threshold_0.8,saliency_rare,1.380000,1.090000
6,unirare_finetune_threshold_0.8,saliency_fusion_add,0.720000,1.460000
7,unirare_finetune_threshold_0.8,saliency_fusion_rs,1.000000,1.190000
8,unirare_exp_finetune_threshold_0.7,saliency,0.130000,31.500000
9,unirare_exp_finetune_threshold_0.7,saliency_rare,1.580000,1.130000


P3_data_orientations


,Models,Type,MSRT,MSRB
0,unirare_exp2_finetune_threshold_0.8,saliency,0.140000,61.710000
1,unirare_exp2_finetune_threshold_0.8,saliency_rare,2.910000,1.030000
2,unirare_exp2_finetune_threshold_0.8,saliency_fusion_add,0.880000,1.240000
3,unirare_exp2_finetune_threshold_0.8,saliency_fusion_rs,1.010000,1.150000
4,unirare_finetune_threshold_0.8,saliency,0.140000,37.730000
5,unirare_finetune_threshold_0.8,saliency_rare,2.040000,1.020000
6,unirare_finetune_threshold_0.8,saliency_fusion_add,0.830000,1.290000
7,unirare_finetune_threshold_0.8,saliency_fusion_rs,1.100000,1.120000
8,unirare_exp_finetune_threshold_0.7,saliency,0.140000,38.880000
9,unirare_exp_finetune_threshold_0.7,saliency_rare,2.510000,1.030000
